In [1]:
import re

## Posts
- Id (integer) - id of the post
- PostTypeId (integer: 1 or 2) - 1 for questions, 2 for answers
- CreationDate (date) - post creation date in the format "YYYY-MM-DDTHH:MM:SS.ms"
- Tags (string, optional) - list of post tags, each tag is wrapped with html entities '&lt;' and '&gt;'
- OwnerUserId (integer, optional) - user id of the post's author
- ParentId (integer, optional) - for answers - id of the question
- Score (integer) - score (votes) of a question or an answer, can be negative (!)
- FavoriteCount (integer, optional) - how many times the question was added in the favorites

In [2]:
%%writefile posts.xml
<row Id="11" PostTypeId="1" AcceptedAnswerId="1248" CreationDate="2008-07-31T23:55:37.967" Score="1106" ViewCount="115048" Body="&lt;p&gt;Given a specific &lt;code&gt;DateTime&lt;/code&gt; value, how do I display relative time, like:&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;2 hours ago&lt;/li&gt;&#xA;&lt;li&gt;3 days ago&lt;/li&gt;&#xA;&lt;li&gt;a month ago&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" OwnerUserId="1" LastEditorUserId="1136709" LastEditorDisplayName="user2370523" LastEditDate="2015-12-29T02:08:37.450" LastActivityDate="2016-07-13T23:23:58.537" Title="How can relative time be calculated in C#?" Tags="&lt;c#&gt;&lt;datetime&gt;&lt;datediff&gt;&lt;relative-time-span&gt;" AnswerCount="33" CommentCount="4" FavoriteCount="508" CommunityOwnedDate="2009-09-04T13:15:59.820" />
<row Id="1394" PostTypeId="2" ParentId="1390" CreationDate="2008-08-04T16:38:03.667" Score="16" Body="&lt;p&gt;Not sure how credible &lt;a href=&quot;http://www.builderau.com.au/program/windows/soa/Getting-started-with-Windows-Server-2008-Core-edition/0,339024644,339288700,00.htm&quot; rel=&quot;nofollow noreferrer&quot;&gt;this source is&lt;/a&gt;, but:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;The Windows Server 2008 Core edition can:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the file server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Hyper-V virtualization server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Directory Services role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DHCP server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the IIS Web server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DNS server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Active Directory Lightweight Directory Services.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the print server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;/ul&gt;&#xA;  &#xA;  &lt;p&gt;The Windows Server 2008 Core edition cannot:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run a SQL Server.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run an Exchange Server.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Internet Explorer.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Windows Explorer.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Host a remote desktop session.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run MMC snap-in consoles locally.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;/ul&gt;&#xA;&lt;/blockquote&gt;&#xA;" OwnerUserId="91" LastEditorUserId="1" LastEditorDisplayName="Jeff Atwood" LastEditDate="2008-08-27T13:02:50.273" LastActivityDate="2008-08-27T13:02:50.273" CommentCount="1" />	
<row Id="3543" PostTypeId="2" ParentId="3530" CreationDate="2008-08-06T15:24:00.787" Score="37" Body="&lt;p&gt;from &lt;a href=&quot;http://www.timocracy.com/articles/2008/02/21/calling-invoking-rails-rake-tasks-from-within-ruby-for-testing-try-2&quot; rel=&quot;nofollow noreferrer&quot;&gt;timocracy.com&lt;/a&gt;:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;require 'rake'&#xA;require 'rake/rdoctask'&#xA;require 'rake/testtask'&#xA;require 'tasks/rails'&#xA;&#xA;def capture_stdout&#xA;  s = StringIO.new&#xA;  oldstdout = $stdout&#xA;  $stdout = s&#xA;  yield&#xA;  s.string&#xA;ensure&#xA;  $stdout = oldstdout&#xA;end&#xA;&#xA;Rake.application.rake_require '../../lib/tasks/metric_fetcher'&#xA;results = capture_stdout {Rake.application['metric_fetcher'].invoke}&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;" OwnerUserId="399" LastActivityDate="2008-08-06T15:24:00.787" CommentCount="3" />

Overwriting posts.xml


In [3]:
posts = []
with open('posts.xml', 'r') as xml:
    for line in xml:
        posts.append(line.strip())
posts[0]

'<row Id="11" PostTypeId="1" AcceptedAnswerId="1248" CreationDate="2008-07-31T23:55:37.967" Score="1106" ViewCount="115048" Body="&lt;p&gt;Given a specific &lt;code&gt;DateTime&lt;/code&gt; value, how do I display relative time, like:&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;2 hours ago&lt;/li&gt;&#xA;&lt;li&gt;3 days ago&lt;/li&gt;&#xA;&lt;li&gt;a month ago&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" OwnerUserId="1" LastEditorUserId="1136709" LastEditorDisplayName="user2370523" LastEditDate="2015-12-29T02:08:37.450" LastActivityDate="2016-07-13T23:23:58.537" Title="How can relative time be calculated in C#?" Tags="&lt;c#&gt;&lt;datetime&gt;&lt;datediff&gt;&lt;relative-time-span&gt;" AnswerCount="33" CommentCount="4" FavoriteCount="508" CommunityOwnedDate="2009-09-04T13:15:59.820" />'

In [4]:
row = '^<row.*? '
Id_ = '(?=.*\\bId=\"(\\d+)\")'
PostTypeId_ = '(?=.*\\bPostTypeId=\"(\\d+)\")'
CreationDate_ = '(?=.*\\bCreationDate=\"([^"]*)\")'
Score_ = '(?=.*\\bScore=\"(-?\\d+)\")'
OwnerUserId_ = '(?=.*\\bOwnerUserId=\"(\\d+)\")?'
ParentId_ = '(?=.*\\bParentId=\"(\\d+)\")?'
FavoriteCount_ = '(?=.*\\bFavoriteCount=\"(\\d+)\")?'

In [5]:
for post in posts:
    print re.search(
        row +
        Id_ + 
        PostTypeId_ +
        CreationDate_ +
        Score_ +
        ParentId_ +
        OwnerUserId_ +
        FavoriteCount_,
            post).groups()

('11', '1', '2008-07-31T23:55:37.967', '1106', None, '1', '508')
('1394', '2', '2008-08-04T16:38:03.667', '16', '1390', '91', None)
('3543', '2', '2008-08-06T15:24:00.787', '37', '3530', '399', None)


## Users
- Id (integer) - user id
- Reputation (integer) - user's reputation
- CreationDate (string) - creation date in the format "YYYY-MM-DDTHH:MM:SS.ms"
- DisplayName (string) - user's name
- Location (string, options) - user's country
- Age (integer, optional) - user's age

In [6]:
%%writefile users.xml
<row Id="4" Reputation="26638" CreationDate="2008-07-31T14:22:31.317" DisplayName="Joel Spolsky" LastAccessDate="2016-12-10T22:12:46.367" WebsiteUrl="http://www.joelonsoftware.com/" Location="New York, NY" AboutMe="&lt;p&gt;I am:&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;the co-founder and CEO of &lt;a href=&quot;http://stackexchange.com">Stack Exchange&lt;/a&gt;&lt;/li&gt;&#xA;&lt;li&gt;the co-founder of &lt;a href=&quot;http://www.fogcreek.com" rel=&quot;nofollow&quot;&gt;Fog Creek Software&lt;/a&gt;&lt;/li&gt;&#xA;&lt;li&gt;the creator and chairman of the board of &lt;a href=&quot;http://trello.com" rel=&quot;nofollow&quot;&gt;Trello&lt;/a&gt;&lt;/li&gt;&#xA;&lt;li&gt;owner of Taco, the most famous Siberian Husky on the Upper West Side.&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;&#xA;&lt;p&gt;You can find me on Twitter (as &lt;a href=&quot;http://twitter.com/spolsky" rel=&quot;nofollow&quot;&gt;@spolsky&lt;/a&gt;) or on my rarely-updated blog, &lt;a href=&quot;http://joelonsoftware.com" rel=&quot;nofollow&quot;&gt;Joel on Software&lt;/a&gt;.&lt;/p&gt;&#xA;" Views="66761" UpVotes="779" DownVotes="96" ProfileImageUrl="https://i.stack.imgur.com/C5gBG.jpg?s=128&g=1" AccountId="4" />
<row Id="14357" Reputation="72371" CreationDate="2008-09-16T23:23:24.623" DisplayName="spender" LastAccessDate="2016-12-10T14:25:39.257" WebsiteUrl="http://radiotuna.com" Location="United Kingdom" AboutMe="&lt;p&gt;Tea afficionado, A.I. graduate, &lt;a href=&quot;https://bugzilla.mozilla.org/show_bug.cgi?id=869725&quot; rel=&quot;nofollow&quot;&gt;Mozillian&lt;/a&gt;, company director and CTO at &lt;a href=&quot;http://radiotuna.com&quot; rel=&quot;nofollow&quot;&gt;radiotuna.com&lt;/a&gt;. Come and listen!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I give my time here because I get so much more in return.&lt;/p&gt;&#xA;" Views="5810" UpVotes="2026" DownVotes="1056" ProfileImageUrl="https://i.stack.imgur.com/BGshb.png" Age="43" AccountId="8089" />	
<row Id="17516" Reputation="145889" CreationDate="2008-09-18T10:29:51.940" DisplayName="AnthonyWJones" LastAccessDate="2016-11-22T13:49:03.680" WebsiteUrl="http://geekswithblogs.net/codingbloke" Location="Birmingham, United Kingdom" AboutMe="&lt;p&gt;&lt;p&gt;General interests: C#, Javascript, Agile, Interaction Design&lt;/p&gt;&#xA;&#xA;&lt;p&gt;&lt;p&gt;Current interests:  Silverlight+Toolkit, Windows Phone 7.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;&lt;p&gt;Would like to know more about: WCF, EF, WPF, XNA, SQL 2008, ... the list is endless.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;&lt;p&gt;First to gain the Silverlight badges, now at Gold.&lt;/p&gt;&#xA;" Views="12801" UpVotes="5757" DownVotes="233" Age="50" AccountId="9467" />	
<row Id="17693" Reputation="16635" CreationDate="2008-09-18T13:46:50.907" DisplayName="Matt" LastAccessDate="2016-12-08T17:16:17.217" WebsiteUrl="http://www.squaregear.net/" Location="United States" AboutMe="&lt;p&gt;I'm just some guy who does some programming stuff.&lt;/p&gt;&#xA;" Views="832" UpVotes="109" DownVotes="0" ProfileImageUrl="https://i.stack.imgur.com/JgeMc.png" Age="44" AccountId="9547" />	

Overwriting users.xml


In [7]:
users = []
with open('users.xml', 'r') as xml:
    for line in xml:
        users.append(line.strip())
users[0]

'<row Id="4" Reputation="26638" CreationDate="2008-07-31T14:22:31.317" DisplayName="Joel Spolsky" LastAccessDate="2016-12-10T22:12:46.367" WebsiteUrl="http://www.joelonsoftware.com/" Location="New York, NY" AboutMe="&lt;p&gt;I am:&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;the co-founder and CEO of &lt;a href=&quot;http://stackexchange.com">Stack Exchange&lt;/a&gt;&lt;/li&gt;&#xA;&lt;li&gt;the co-founder of &lt;a href=&quot;http://www.fogcreek.com" rel=&quot;nofollow&quot;&gt;Fog Creek Software&lt;/a&gt;&lt;/li&gt;&#xA;&lt;li&gt;the creator and chairman of the board of &lt;a href=&quot;http://trello.com" rel=&quot;nofollow&quot;&gt;Trello&lt;/a&gt;&lt;/li&gt;&#xA;&lt;li&gt;owner of Taco, the most famous Siberian Husky on the Upper West Side.&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;&#xA;&lt;p&gt;You can find me on Twitter (as &lt;a href=&quot;http://twitter.com/spolsky" rel=&quot;nofollow&quot;&gt;@spolsky&lt;/a&gt;) or on my rarely-updated blog, &lt;a href=&quot;http://joelonsoftware.com" rel=

In [8]:
row = '^<row.*? '
id_ = '(?=.*\\bId=\"(\\d+)\")'
Reputation_ = '(?=.*\\bReputation=\"(\\d+)\")'
CreationDate_ = '(?=.*\\bCreationDate=\"([^"]*)\")'
DisplayName_ = '(?=.*\\bDisplayName=\"([^"]*)\")'
Location_ = '(?=.*\\bLocation=\"([^"]*)\")?'
Age_ = '(?=.*\\bAge=\"(\\d+)\")?'

In [9]:
for user in users:
    print re.search(
        row +
        Id_ + 
        Reputation_ +
        CreationDate_ +
        DisplayName_ +
        Location_ +
        Age_,
            user).groups()

('4', '26638', '2008-07-31T14:22:31.317', 'Joel Spolsky', 'New York, NY', None)
('14357', '72371', '2008-09-16T23:23:24.623', 'spender', 'United Kingdom', '43')
('17516', '145889', '2008-09-18T10:29:51.940', 'AnthonyWJones', 'Birmingham, United Kingdom', '50')
('17693', '16635', '2008-09-18T13:46:50.907', 'Matt', 'United States', '44')
